In [2]:
## SETUP 
imports = ['wrds', 'pandas as pd', 'os', 're', 'pickle', 'numpy as np', 'from name_matching.name_matcher import NameMatcher',
          'from joblib import Parallel, delayed', 'from IPython.display import display, HTML, clear_output',
          'unicodedata', 'sys']
for command in imports:
    if command.startswith('from'): exec(command)
    else: exec('import ' + command)

if not os.getcwd().endswith('Big Data'):
    os.chdir('../..')

sys.path.append('trade_data_code/2_python')
import A_helper_functions as hf



NOTE THAT THE FOLLOWING SECTION WON'T RUN BC I'VE 

In [3]:
########################################################################################
# USE ROLE DATA TO DETERMINE WHICH FIRMS ARE LIKELY FRENCH 
########################################################################################

french_leis = pd.read_parquet('data/2_processed/admin/LEI_siren_crosswalk.parquet')['lei'].unique()
non_french_country_domains = (pd.read_excel('data/2_processed/admin/domain_names_by_country.xlsx')
                              .assign(name=lambda x: x['name'].str.replace('.', '', regex=False))
                              .query('include != include')['name'])
french_factset_ids = pd.read_parquet('data/2_processed/admin/factset_french_domiciled.parquet')['factset_entity_id'].unique()

companies = (
    ## determine the firm's max total / french / data values and shares 
    pd.read_parquet('data/2_processed/linkedin/french_affiliated_firm_roles_collapsed_raw.parquet')
    .groupby('rcid', as_index=False)
    .agg({'emp_total': 'max','emp_french': 'max', 'share_emp_french': 'max', 'share_comp_french': 'max', 'emp_data': 'max'})
    .assign(french_eligible = lambda c: c['emp_french'].gt(0),
         data_eligible = lambda c: c['emp_data'].gt(0))
    .assign(role_eligible = lambda c: c['data_eligible'] & c['french_eligible'])
    [['rcid', 'french_eligible', 'data_eligible','role_eligible', 'share_emp_french', 'share_comp_french']].
    merge(pd.read_parquet('data/1_raw_data/linkedin/revelio/france_affiliated_firms.parquet'))
    
    ### determine whether the firm likely french or not 
    .assign(url_ending = lambda c: c['url'].apply(lambda x: x.split('.')[-1] if isinstance(x, str) else None))
    .assign(
        admin_score=lambda c: 0
        ## TOP LEVEL DOMAIN 
        + c['url_ending'].eq('fr')  
        - c['url_ending'].isin(non_french_country_domains)  

        ## FACTSET
        + c['factset_entity_id'].isin(french_factset_ids)
        - (~c['factset_entity_id'].isin(french_factset_ids) & c['factset_entity_id'].notna())

        # LEI CHECK 
        +  c['lei'].isin(french_leis) 
        -  (~c['lei'].isin(french_leis) & c['lei'].notna()) 

        # ISIN 
        + c['isin'].str[:2].eq("FR") # add if french isin
        - (~c['isin'].str[:2].eq("FR") & c['isin'].notna())

        # CUSIP 
        + c['cusip'].str[:1].eq("F") # add if french cusip
        - (~c['cusip'].str[:1].eq("F") & c['cusip'].notna())

        # Firm type 
        + c['firm_type_french_likelihood'].eq("likely french") 
        - c['firm_type_french_likelihood'].eq("unlikely french"))
     .assign(
         likely_french = lambda c: 
         c['french_eligible'] & (
         c['admin_score'].gt(0) | 
         (c['admin_score'].eq(0) & (c['share_emp_french'].gt(.5) | c['share_comp_french'].gt(.5)))))
    
    #### mark whether it's a subsidiary 
    .assign(subsidiary = lambda c: c['rcid'] != c['ultimate_parent_rcid'])
    [['rcid','lei','company','company_cleaned', 'company_stripped', 'year_founded', 'ultimate_parent_rcid',
      'french_eligible', 'data_eligible', 'role_eligible', 'likely_french', 'subsidiary']]
)
companies.to_parquet('data/2_processed/linkedin/france_affiliated_firms_cleaned.parquet')
